In [1]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install jiwer -q
!pip install sacrebleu -q
!pip install pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [2]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, Dataset, Audio, DatasetDict, concatenate_datasets
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import pandas as pd


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from huggingface_hub import whoami

# Get user info
user_info = whoami()
print(user_info)

{'type': 'user', 'id': '6573e3d7244aefdfc476f07a', 'name': 'cobrayyxx', 'fullname': 'M Hazim AF', 'email': 'muhammadhazim86@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/16ef7a7d5351781180384c7f5b4ef392.svg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '67a769f44edabdd17b5c169b', 'name': 'SpeechT', 'fullname': 'SpeechT', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/618c42e4f35289d4bcab9cf1/m1k2sALFEwa9B2McXNyOi.png', 'roleInOrg': 'read', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'colab2', 'role': 'wri

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
# train_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025",split="train+val", streaming=True)
train_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025",split="train", streaming=True)

README.md:   0%|          | 0.00/620 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [10]:
test_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025", split="test", streaming=True)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [11]:
train_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 41
})

In [12]:
test_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 2
})

# Testing with first 10 data (this is just for checking whether the training code is working or not)

In [13]:
# Get the first 10 examples
train_dataset = list(train_dataset.take(10))
test_dataset = list(test_dataset.take(10))

In [14]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [15]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

In [16]:
test_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 10
})

# Preprocess Data

In [17]:
train_dataset = train_dataset.remove_columns(["audio", "speaker_id"])  # Remove audio
test_dataset = test_dataset.remove_columns(["audio", "speaker_id"])

In [18]:
train_dataset

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 10
})

In [19]:
test_dataset

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 10
})

In [20]:
df_train = train_dataset.to_pandas()
df_test = test_dataset.to_pandas()

In [21]:
df_train.shape

(10, 2)

In [22]:
df_test.shape

(10, 2)

In [23]:
df_train = df_train.drop_duplicates(subset=["sentence"], keep="first").reset_index(drop=True)
df_test = df_test.drop_duplicates(subset=["sentence"], keep="first").reset_index(drop=True)

In [24]:
df_train.shape

(10, 2)

In [25]:
df_test.shape

(10, 2)

In [26]:
df_train.head()

,sentence,translation
0,Abalumendo babili nabeminina bale ikata amabul...,these men are standing caching tree leaves
1,Cifwile uyu ou bali nankwe eulemulanga ifyakuc...,It must be that the person they are with is sh...
2,Umulumendo ekele pacipuna elo kumaboko ekete i...,A young man sitted on the chair has a red doll...
3,Shili mu musebo,They are on the road.
4,Umwaume uwabuuta inkanda aletekenya icimbaya m...,One white man is driving a tractor


## remove null

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence     10 non-null     object
 1   translation  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes


In [28]:
df_train = df_train[~df_train["translation"].isna()]

In [29]:
df_train.shape

(10, 2)

In [30]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence     10 non-null     object
 1   translation  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes


In [31]:
cleaned_train = Dataset.from_pandas(df_train)
cleaned_test = Dataset.from_pandas(df_test)

In [32]:
cleaned_train

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 10
})

In [33]:
cleaned_test

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 10
})

In [34]:
len(cleaned_train["translation"])

10

# Add Additional Data

In [54]:
train_df = pd.read_csv("bt_tatoeba_en_bem_2.csv")

In [55]:
train_df.shape

(85154, 2)

In [57]:
train_df.head()

,text_bem,text_en
0,"<bt> Tekulanda saana, citafye ifingi.","Talk less, do more."
1,<bt> Ukwesha kwena cintu icisuma saana.,Trying is something really beautiful.
2,<bt> awe ine nshikwetabila iyo.,I no longer need you.
3,<bt> Ndemona kwati apa ndesubila ndeya kusukulu.,I am going to school right now.
4,<bt> Awe kwena kuti bakwipaya.,They'll kill you!


## remove duplicate (optional)

In [ ]:
df_duplicate = train_df[train_df.duplicated(subset=["text_bem"])]

In [ ]:
train_df[train_df["text_bem"] == "<bt> Banina, naumfwa insala."]

In [ ]:
train_df_no_duplicate = train_df.drop_duplicates(subset=["text_bem"], keep="first").reset_index(drop=True)

,text_en,text_bem
118,"Mum, I am hungry.","<bt> Banina, naumfwa insala."


In [ ]:
train_df = train_df_no_duplicate

,text_en,text_bem
64,"Mom, I am hungry.","<bt> Banina, naumfwa insala."
118,"Mum, I am hungry.","<bt> Banina, naumfwa insala."


In [ ]:
train_df_no_duplicate.shape

## concatenate Dataset

In [58]:
train_dataset_augmented = Dataset.from_pandas(train_df)

In [59]:
train_dataset_augmented

Dataset({
    features: ['text_bem', 'text_en'],
    num_rows: 85154
})

In [60]:
train_dataset_augmented = train_dataset_augmented.rename_column("text_en", "translation")

In [61]:
train_dataset_augmented = train_dataset_augmented.rename_column("text_bem", "sentence")

In [62]:
train_dataset_augmented

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 85154
})

In [74]:
train_dataset_concat = concatenate_datasets([train_dataset, train_dataset_augmented])

In [75]:
train_dataset_concat

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 85164
})

In [76]:
df_train = train_dataset_concat.to_pandas()

In [77]:
df_train.shape

(85164, 2)

In [81]:
df_train.tail()

,sentence,translation
85159,<bt> Cilemoneka kwati caliba icilubo icikalamb...,That would be a huge mistake.
85160,<bt> Bushe wali sambilila icilatini ku sukulu?,Have you been learning Latin at school?
85161,<bt> Nipundile ifyakucita.,Tell me what we should do.
85162,<bt> Twakonka ifyo batwebele elyo twapwisha um...,We followed his instructions and finished the ...
85163,<bt> Pantu twaleikala twapalamana saana.,We lived next to each other.


In [86]:
# shuffle the row
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

In [91]:
df_train.head()

,sentence,translation
0,<bt> awe naine naumfwa ilyashi lya uyu umusango.,I've also heard such a story.
1,<bt> Ukuloba isabi mu mumana balilesha.,Fishing in this lake is prohibited.
2,<bt> Bushe naubelenga icitenge na icitenge?,Have you read it cover to cover?
3,"<bt> Icimbayambaya nga caonaika, ndefwaika nji...","If the car breaks down, I’ll have to go by sha..."
4,<bt> Ala fyonse filacinja bwangu saana.,Everything changes very quickly.


In [92]:
df_train.tail()

,sentence,translation
85159,"<bt> Ukulufyanyampwa nga kwaba amafunde, ulufy...","When injustice becomes law, resistance becomes..."
85160,"<bt> Pakutampa ukukopolola, tinikapo batani ly...","When you begin downloading, please press the ""..."
85161,<bt> Tewakupanga ukuti nacikupapusha iyo.,Don't pretend to be interested in it.
85162,<bt> awe kwena teti njishibe ukupilibula ishi ...,I don't know how to translate this word.
85163,"<bt> Emukwai nifyofine, elo uyu umulumendo ale...",Tom receives a disability pension.


In [103]:
# cleaned_train = Dataset.from_pandas(df_train.head(10)) # For testing purpose
cleaned_train = Dataset.from_pandas(df_train)

In [104]:
cleaned_train

Dataset({
    features: ['sentence', 'translation'],
    num_rows: 10
})

# Training

In [95]:
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")
metric_wer = evaluate.load("wer")

In [96]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    bleu = round(bleu["score"], 2)

    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"bleu": bleu, "chrf": chrf, "wer": wer}

In [97]:
sentences = cleaned_train["sentence"][:5]

# Set the language tokens
tokenizer.src_lang = "bem_Latn"  # Source language: eng in Latin script
tokenizer.tgt_lang = "eng_Latn"  # Target language: hin in Latin script

# Tokenize and set source/target languages
inputs = tokenizer(
    sentences,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Generate translations
translated = model.generate(**inputs)
translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Print translations
for i, translation in enumerate(translations):
    print(f"Input: {sentences[i]}")
    print(f"Translation: {translation}\n")

Input: <bt> awe naine naumfwa ilyashi lya uyu umusango.
Translation: I've heard of this type of story.

Input: <bt> Ukuloba isabi mu mumana balilesha.
Translation: Fishing in the river is prohibited.

Input: <bt> Bushe naubelenga icitenge na icitenge?
Translation: <bt> Have you read the ceiling and ceiling?

Input: <bt> Icimbayambaya nga caonaika, ndefwaika njingaleko pa shanki.
Translation: If the carriage is damaged, I need a seat on the tank.

Input: <bt> Ala fyonse filacinja bwangu saana.
Translation: Everything changes quickly.



In [105]:
def preprocess_function(dataset):
    # Tokenize the source text (English) and target text (Hindi)
    try:
      model_inputs = tokenizer(
          dataset["sentence"],  # Source text column
          text_target=dataset["translation"],  # Target text column
          max_length=128,  # Adjust max length based on your needs
          truncation=True,  # Truncate inputs that exceed the max length
          padding="max_length"  # Pad inputs to the max length
      )
      return model_inputs
    except Exception as e:
      print(e)

In [106]:
train_tokenized = cleaned_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [107]:
test_tokenized = cleaned_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [108]:
train_tokenized

Dataset({
    features: ['sentence', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [109]:
test_tokenized

Dataset({
    features: ['sentence', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [110]:
# adjust the variable below
hf_repo = "kreasof-ai"
output_dir = "nllb-IWSLT2025-bem-en-dum"

In [111]:
# Fine-tune
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,               # Directory to save checkpoints
    per_device_train_batch_size=16,      # Adjust batch size based on GPU memory
    gradient_accumulation_steps=1,
    learning_rate=1e-4,                   # Learning rate
    warmup_ratio=0.03,
    num_train_epochs=1,                # Train for 100 epochs
    gradient_checkpointing=True,


    fp16=True,
    hub_model_id=f"{hf_repo}/{output_dir}",  # Change this
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch

    per_device_eval_batch_size=8,
    predict_with_generate=True,          # Generate predictions for validation
    generation_max_length=128,

    save_strategy="epoch",               # Save checkpoints at the end of each epoch
    load_best_model_at_end=True,         # Load the best model at the end of training
    metric_for_best_model="chrf",        # Use BLEU as the evaluation metric
    greater_is_better=True,
    logging_dir="./logs",                # Log directory
    logging_steps=100,
    report_to=["tensorboard"],
    push_to_hub=True,                    # Push model to Hugging Face Hub

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [112]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

<ipython-input-112-2ce0fa6046c1>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [113]:
# trainer.train()
# trainer.train(resume_from_checkpoint=True)
# trainer.train(resume_from_checkpoint="./nllb-600/checkpoint-1000") # resume from specific checkpoint
if os.path.exists(output_dir):
    trainer.train(resume_from_checkpoint=True)
else:
    trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Wer
1,No log,12.508872,6.090000,26.730000,88.235294


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1, training_loss=12.875518798828125, metrics={'train_runtime': 87.6364, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.011, 'total_flos': 2708880752640.0, 'train_loss': 12.875518798828125, 'epoch': 1.0})

In [114]:
# Push the final model to Hugging Face Hub
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/kreasof-ai/nllb-IWSLT2025-bem-en-dum/commit/a5cd8e70de310be514c666efc890f68b28589150', commit_message='End of training', commit_description='', oid='a5cd8e70de310be514c666efc890f68b28589150', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kreasof-ai/nllb-IWSLT2025-bem-en-dum', endpoint='https://huggingface.co', repo_type='model', repo_id='kreasof-ai/nllb-IWSLT2025-bem-en-dum'), pr_revision=None, pr_num=None)